In [1]:
from caveclient import CAVEclient
from intern import array
import pickle
import numpy as np
from agents import data_loader
from cloudvolume import CloudVolume
from membrane_detection import membranes
from agents.scripts import precompute_membrane_vectors, create_post_matrix, merge_paths, get_soma
import agents.sensor
from agents.run import run_agents
import aws.sqs as sqs
import sys
import time
import ast
import pandas as pd
import agents.scripts as scripts
from drive import drive
from finding_orphans import *
from math import sqrt

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Testing tip finder on neurons w somas

In [5]:
tl, skel, mesh_obj = tip_finding.tip_finder_decimation(root_id = 864691135489218668, inval_d = 5600, num_humfrey_iters= 200, decimation_factor= 0.9)
 #looking at the tuple list and printing out all of the endpoints   
print("len of tl: ",len(tl))
print()
print("tl: ",tl)
# print()
# print("cc2: ")
# print(len(cc2))
# print(cc2)


08/02/2022 03:43:19 PM WARNING: face_normals incorrect shape, ignoring! 


100%|██████████| 66050/66050 [00:00<00:00, 478007.07it/s]


len of tl:  3

tl:  [(127626, 180942, 21828), (127834, 168512, 22169), (145833, 168943, 22017)]


In [ ]:
cc2_lens = []
for arr in cc2:
    cc2_lens.append(len(arr))

print("Maximum number of connected components: ",max(cc2_lens))
max_index = [index for index, item in enumerate(cc2_lens) if item == max(cc2_lens)]
cc2_lens[max_index[0]] = 0
print("Second largest number of connected components: ",max(cc2_lens))

In [ ]:
unique_vals, counts = np.unique(cc, return_counts = True)
print(counts)

In [ ]:
print(len(edges_by_component))
max_ind = (edges_by_component.argmax())
print(edges_by_component[max_ind])
edges_by_component[max_ind] = 0
print(edges_by_component[max_ind])
second_max_ind = edges_by_component.argmax()
print(edges_by_component[second_max_ind])
# edges_by_componenet[second_max_ind]
# print(len(edges_by_component[edges_by_component > 5]))

In [ ]:
mesh_obj.edges

In [ ]:
vertexA = mesh_obj.vertices[mesh_obj.edges[0][0]]
vertexB = mesh_obj.vertices[mesh_obj.edges[0][1]]
x1 = vertexA[0]
x2 = vertexB[0]
y1 = vertexA[1]
y2 = vertexB[1]
z1 = vertexA[2]
z2 = vertexB[2]
edge_length = sqrt(((x1 - x2)**2) + ((y1-y2)**2) + ((z1-z2)**2))
print(vertexA)
print(vertexB)
print(edge_length)


In [ ]:
from meshparty import trimesh_vtk
mesh_actor = trimesh_vtk.mesh_actor(mesh_obj, opacity=.5)
trimesh_vtk.render_actors([mesh_actor])

In [ ]:
from meshparty import trimesh_vtk
import trimesh


def skel_vis(skel):
    import meshparty.skeleton_io as skel_io
    from meshparty import trimesh_vtk

    mesh_actor = trimesh_vtk.skeleton_actor(skel,
                                            color=(1, 0, 0),
                                            opacity=0.5)

    end_point_actor = trimesh_vtk.point_cloud_actor(skel.vertices[skel.end_points], size=1200, color=(0, 1, 0),
                                                    opacity=1)

    trimesh_vtk.render_actors([mesh_actor, end_point_actor])

skel_vis(skel)

# mesh_obj = trimesh.smoothing.filter_laplacian(mesh_obj, iterations=50)


# mesh_actor = trimesh_vtk.mesh_actor(mesh_obj,opacity=.5)
# trimesh_vtk.render_actors([mesh_actor])



In [ ]:
# # Trying to get all endpoints on mesh_obj

# edges_by_component2 = trimesh.graph.connected_component_labels(
#     mesh_obj.face_adjacency)
# largest_component2 = np.bincount(edges_by_component2).argmax()
# largest_component_size2 = np.sum(edges_by_component2 == largest_component2)

# cc2 = trimesh.graph.connected_components(
#     mesh_obj.face_adjacency, min_len=largest_component_size2-1)

# mask2 = np.zeros(len(mesh_obj.faces), dtype=bool)

# mask2[np.concatenate(cc2)] = True

# mesh_obj.update_faces(mask2)
# skel2 = skeletonize.skeletonize_mesh(trimesh_io.Mesh(mesh_obj.vertices,
#                                                      mesh_obj.faces))
# degree_dict2 = {}
# unique_ids2 = np.unique(skel2.edges)
# for i in unique_ids2:
#     degree_dict2[i] = np.sum(skel2.edges == i)

# degree2 = np.array(list(degree_dict2.values()))
# points2 = skel2.vertices[np.argwhere(degree == 1)].astype(int)
# tl2 = [tuple([int(e[0][0])//4, int(e[0][1])//4, int(e[0][2])//40]) for e in points2]

# print(len(tl2))


### Testing orphan tip finder

In [ ]:
# Importing data

data_path = "/Users/RupaChalavadi/Desktop/15_ng_links_edited_20220610.csv"
data = pd.read_csv(data_path)

In [ ]:
# Cleaning up data
seg_ids_endpoints = data.iloc[:,2:4]
# seg_ids_endpoints = seg_ids_endpoints.to_numpy
for i in range(seg_ids_endpoints.shape[0]):
    endpoints_str = seg_ids_endpoints.iloc[i][0]
    endpoints_list = list(endpoints_str.split(','))
    print(endpoints_list)
    

# seg_ids_endpoints


In [ ]:
# seg_ids = [864691135909994000,
#            864691135247440303,
#            864691134959738482,
#            864691135314714227,
#            864691135552841025,
#            ]
# endpoints = [[402188, 228684, 24029],[401258, 224832, 24029],[401314, 228366, 24424],[400199, 220721, 24029],[397870, 232292, 24004],[403272, 227529, 24394]]

# closest_calculated_endpoints = []


# for i in seg_ids:
#     print(tip_finding.tip_finder_decimation(i))

# # for i in range(len(seg_ids)):
# #     endpoints_list = tip_finding.tip_finder_decimation(seg_ids[i])
#     # distances_list = []
#     # for point in endpoints_list:
#     #     distance = sqrt((point[0] - endpoints[i][0])**2 + (point[1] - endpoints[i][1])**2 + (point[2] - endpoints[i][2])**2)
#     #     distances_list.append(distance)
    
#     # min_index = distances_list.index(min(distances_list))
#     # closest_calculated_endpoints.append(endpoints_list[min_index])

# # closest_calculated_endpoints


    

In [ ]:
tl, skel = tip_finding.tip_finder_decimation(864691135909994000)
